In [3]:
!pwd
!ls -l

/Users/mac/Documents/MASTER_DATA_SCIENCE/tfm
total 307536
-rw-r--r--@  1 mac  staff  114585421 20 mar 08:01 20170319.gkg.csv
-rw-r--r--@  1 mac  staff   34848148 20 mar 08:01 20170319.gkg.csv.zip
-rw-r--r--@  1 mac  staff    7404966 10 feb 21:30 9781783985128-LEARNING_PANDAS.pdf
-rw-r--r--@  1 mac  staff     101351 20 mar 10:51 Basecamp KNN.ipynb
-rw-r--r--@  1 mac  staff     367286 20 mar 10:51 Decision Tree.ipynb
drwx------@  8 mac  staff        272 30 mar 00:16 Diseno_Registro_anterior_2010
drwx------@  7 mac  staff        238 30 mar 00:16 Diseno_Registro_desde_2011
-rw-r--r--@  1 mac  staff     121567 20 mar 10:51 Ensemble.ipynb
drwx------@  5 mac  staff        170 26 mar 10:39 MICRODATOS_ACC_VICT_2008
drwx------@  5 mac  staff        170 26 mar 10:39 MICRODATOS_ACC_VICT_2009
drwx------@  5 mac  staff        170 26 mar 10:40 MICRODATOS_ACC_VICT_2010
drwx------@  5 mac  staff        170 26 mar 10:40 MICRODATOS_ACC_VICT_2011
drwx------@  5 mac  staff        170 26 mar 10:39 MICRODATO

In [16]:
!ls
!pwd

20170319.gkg.csv                  TFM_DGT.ipynb
20170319.gkg.csv.zip              Untitled1.ipynb
9781783985128-LEARNING_PANDAS.pdf metadata
Basecamp KNN.ipynb                raw
Decision Tree.ipynb               raw_zip
Ensemble.ipynb
/Users/mac/Documents/MASTER_DATA_SCIENCE/tfm


#### IMPORTO LIBRERIAS ####

In [1]:
import pandas as pd
from sklearn import linear_model

In [2]:
!ls -l


total 308064
-rw-r--r--@  1 mac  staff  114585421 20 mar 08:01 20170319.gkg.csv
-rw-r--r--@  1 mac  staff   34848148 20 mar 08:01 20170319.gkg.csv.zip
-rw-r--r--@  1 mac  staff    7404966 10 feb 21:30 9781783985128-LEARNING_PANDAS.pdf
-rw-r--r--@  1 mac  staff     101351 20 mar 10:51 Basecamp KNN.ipynb
-rw-r--r--@  1 mac  staff     367286 20 mar 10:51 Decision Tree.ipynb
-rw-r--r--@  1 mac  staff     121567 20 mar 10:51 Ensemble.ipynb
-rw-------@  1 mac  staff      34582  6 abr 16:24 Libro1.xlsx
-rw-r--r--   1 mac  staff     106672  6 abr 16:24 TFM_DGT-pruebas.ipynb
-rw-r--r--   1 mac  staff     133090  6 abr 16:24 TFM_DGT.ipynb
-rw-r--r--   1 mac  staff       4654 22 mar 12:29 Untitled1.ipynb
-rw-r--r--   1 mac  staff        909  6 abr 11:31 libro1.csv
drwxr-xr-x   5 mac  staff        170 30 mar 23:28 metadata
drwxr-xr-x  11 mac  staff        374 30 mar 23:28 raw
drwxr-xr-x  12 mac  staff        408 26 mar 10:42 raw_zip


In [93]:
!csvlook libro1.csv>gg

/bin/sh: $gg: ambiguous redirect


In [3]:
files=!ls -d raw/MICRODATOS_ACC_VICT_20*
files

['raw/MICRODATOS_ACC_VICT_2008',
 'raw/MICRODATOS_ACC_VICT_2009',
 'raw/MICRODATOS_ACC_VICT_2010',
 'raw/MICRODATOS_ACC_VICT_2011',
 'raw/MICRODATOS_ACC_VICT_2012',
 'raw/MICRODATOS_ACC_VICT_2013',
 'raw/MICRODATOS_ACC_VICT_2014',
 'raw/MICRODATOS_ACC_VICT_2015']

#### GUARDO UN LISTADO POR TIPOLOGIA DE FICHERO, PARA POSTERIORMENTE ALMACENARLO EN UN DICCIONARIO ###

In [4]:
accvict=!ls ./raw/MICRODATOS_ACC_VICT_20*/TABLA_ACCVICT_*.csv
pers=!ls ./raw/MICRODATOS_ACC_VICT_20*/TABLA_PERS_*.csv
vehic=!ls ./raw/MICRODATOS_ACC_VICT_20*/TABLA_VEHIC_*.csv
dic_all_files={'Accvict':accvict,'Pers':pers,'Vehic':vehic}
len(dic_all_files)

3

#### RUTAS DE TODOS LOS FICHEROS ####

In [7]:
dic_all_files

{'Accvict': ['./raw/MICRODATOS_ACC_VICT_2008/TABLA_ACCVICT_24H_2008.csv',
  './raw/MICRODATOS_ACC_VICT_2009/TABLA_ACCVICT_24H_2009.csv',
  './raw/MICRODATOS_ACC_VICT_2010/TABLA_ACCVICT_24H_2010.csv',
  './raw/MICRODATOS_ACC_VICT_2011/TABLA_ACCVICT_2011.csv',
  './raw/MICRODATOS_ACC_VICT_2012/TABLA_ACCVICT_2012.csv',
  './raw/MICRODATOS_ACC_VICT_2013/TABLA_ACCVICT_2013.csv',
  './raw/MICRODATOS_ACC_VICT_2014/TABLA_ACCVICT_2014.csv',
  './raw/MICRODATOS_ACC_VICT_2015/TABLA_ACCVICT_2015.csv'],
 'Pers': ['./raw/MICRODATOS_ACC_VICT_2008/TABLA_PERS_24H_2008.csv',
  './raw/MICRODATOS_ACC_VICT_2009/TABLA_PERS_24H_2009.csv',
  './raw/MICRODATOS_ACC_VICT_2010/TABLA_PERS_24H_2010.csv',
  './raw/MICRODATOS_ACC_VICT_2011/TABLA_PERS_2011.csv',
  './raw/MICRODATOS_ACC_VICT_2012/TABLA_PERS_2012.csv',
  './raw/MICRODATOS_ACC_VICT_2013/TABLA_PERS_2013.csv',
  './raw/MICRODATOS_ACC_VICT_2014/TABLA_PERS_2014.csv',
  './raw/MICRODATOS_ACC_VICT_2015/TABLA_PERS_2015.csv'],
 'Vehic': ['./raw/MICRODATOS_ACC_VI

### Ahora que los datos estan trazados, procedemos a realizar una observacion preliminar para comprobar si hay homgeneidad en los datos, y si disponemos de la misma informacion en cada data set, ya que segun la DGT, las extruturas de los datos cambian a partir del 2011 en adelante.

In [35]:
#Obtengo la cabecera del fichero de Accidentes:

## 2008 ##

head_acc_2008=pd.read_csv(dic_all_files['Accvict'][0],delimiter=';',nrows=0,encoding="ISO-8859-1")
head_acc_2008=head_acc_2008.transpose()
head_acc_2008.reset_index(inplace=True)
head_acc_2008.columns=["head_acc_2008"]

head_acc_2015=pd.read_csv(dic_all_files['Accvict'][7],delimiter=';',index_col=False,nrows=0,encoding="ISO-8859-1")
head_acc_2015=head_acc_2015.transpose()
head_acc_2015.reset_index(inplace=True)
head_acc_2015.columns=["head_acc_2015"]
head_acc_2015

,head_acc_2008
0,ID_ACCIDENTE
1,ANIO
2,MES
3,HORA
4,DIASEMANA
5,PROVINCIA
6,COMUNIDAD_AUTONOMA
7,ISLA
8,MUNICIPIO
9,TOT_VICTIMAS


In [63]:
headers={}
for key in dic_all_files.keys():
    for year in [0,7]:
        if year==0:
            df_2008=pd.read_csv(dic_all_files[key][year],delimiter=';',nrows=0,encoding="ISO-8859-1")
            df_2008=df_2008.transpose()
            df_2008.reset_index(inplace=True)
            df_2008.columns=[key+'_2008']
        else:
            df_2015=pd.read_csv(dic_all_files[key][year],delimiter=';',nrows=0,encoding="ISO-8859-1")
            df_2015=df_2015.transpose()
            df_2015.reset_index(inplace=True)
            df_2015.columns=[key+'_2015']
        
    headers={key:df_2008.merge(df_2015, left_on=key+'_2008', right_on=key+'_2015', how='outer')}

In [64]:
Headers


{'Accvict':                 Accvict_2008              Accvict_2015
 0               ID_ACCIDENTE              ID_ACCIDENTE
 1                       ANIO                      ANIO
 2                        MES                       MES
 3                       HORA                      HORA
 4                  DIASEMANA                 DIASEMANA
 5                  PROVINCIA                 PROVINCIA
 6         COMUNIDAD_AUTONOMA        COMUNIDAD_AUTONOMA
 7                       ISLA                      ISLA
 8                  MUNICIPIO                 MUNICIPIO
 9               TOT_VICTIMAS              TOT_VICTIMAS
 10               TOT_MUERTOS               TOT_MUERTOS
 11        TOT_HERIDOS_GRAVES        TOT_HERIDOS_GRAVES
 12         TOT_HERIDOS_LEVES         TOT_HERIDOS_LEVES
 13  TOT_VEHICULOS_IMPLICADOS  TOT_VEHICULOS_IMPLICADOS
 14                      ZONA                      ZONA
 15             ZONA_AGRUPADA             ZONA_AGRUPADA
 16                 CARRETERA        

In [54]:
#head_acc_2008.merge(head_acc_2015, left_on='head_acc_2008', right_on='head_acc_2015', how='outer')
Headers={'Accvict':head_acc_2008.merge(head_acc_2015, left_on='head_acc_2008', right_on='head_acc_2015', how='outer')}

In [55]:
Headers

{'Accvict':                head_acc_2008             head_acc_2015
 0               ID_ACCIDENTE              ID_ACCIDENTE
 1                       ANIO                      ANIO
 2                        MES                       MES
 3                       HORA                      HORA
 4                  DIASEMANA                 DIASEMANA
 5                  PROVINCIA                 PROVINCIA
 6         COMUNIDAD_AUTONOMA        COMUNIDAD_AUTONOMA
 7                       ISLA                      ISLA
 8                  MUNICIPIO                 MUNICIPIO
 9               TOT_VICTIMAS              TOT_VICTIMAS
 10               TOT_MUERTOS               TOT_MUERTOS
 11        TOT_HERIDOS_GRAVES        TOT_HERIDOS_GRAVES
 12         TOT_HERIDOS_LEVES         TOT_HERIDOS_LEVES
 13  TOT_VEHICULOS_IMPLICADOS  TOT_VEHICULOS_IMPLICADOS
 14                      ZONA                      ZONA
 15             ZONA_AGRUPADA             ZONA_AGRUPADA
 16                 CARRETERA        

In [36]:
!csvcut -n -e "ISO-8859-1" $f



  1: ID_ACCIDENTE;ANIO;MES;HORA;DIASEMANA;PROVINCIA;COMUNIDAD_AUTONOMA;ISLA;MUNICIPIO;TOT_VICTIMAS;TOT_MUERTOS;TOT_HERIDOS_GRAVES;TOT_HERIDOS_LEVES;TOT_VEHICULOS_IMPLICADOS;ZONA;ZONA_AGRUPADA;CARRETERA;RED_CARRETERA;TIPO_VIA;TRAZADO_NO_INTERSEC;TIPO_INTERSEC;ACOND_CALZADA;PRIORIDAD;SUPERFICIE_CALZADA;LUMINOSIDAD;FACTORES_ATMOSFERICOS;VISIBILIDAD_RESTRINGIDA;OTRA_CIRCUNSTANCIA;ACERAS;TIPO_ACCIDENTE;DENSIDAD_CIRCULACION;MEDIDAS_ESPECIALES


### PROCEDO A LA CARGAR EN DATAFRAME DE CADA UNO DE LOS FICHEROS DEL DICCIONARIO


In [33]:
#Creo un dataframe vacio para posteriormente rellenarlo con los años 2001 al 2015
#Recorriendo por cada key los n values del diccionario

dic_total={}
df_total=pd.DataFrame()

for key in list_keys_dic:
    print ("#### "+key+" ###")
    for file in dic_all_files[key]:
        #meto en df cada file
        df_year=pd.read_csv(file,delimiter=";",encoding="ISO-8859-1")
        df_year.fillna(0)
        print("Cargando fichero-->"+file+"\n\t"+"Registros= "+str(df_year.shape[0])+"\n\t"+"Columnas= "+str(df_year.shape[1]))
        #df_total=df_total.append(df_year)
        df_total=pd.concat([df_total,df_year],ignore_index=True)
    dic_total[key] = df_total

#### Vehic ###
Cargando fichero-->./raw/MICRODATOS_ACC_VICT_2008/TABLA_VEHIC_24H_2008.csv
	Registros= 163785
	Columnas= 10
Cargando fichero-->./raw/MICRODATOS_ACC_VICT_2009/TABLA_VEHIC_24H_2009.csv
	Registros= 154013
	Columnas= 10
Cargando fichero-->./raw/MICRODATOS_ACC_VICT_2010/TABLA_VEHIC_24H_2010.csv
	Registros= 149277
	Columnas= 10
Cargando fichero-->./raw/MICRODATOS_ACC_VICT_2011/TABLA_VEHIC_2011.csv
	Registros= 144274
	Columnas= 14
Cargando fichero-->./raw/MICRODATOS_ACC_VICT_2012/TABLA_VEHIC_2012.csv
	Registros= 143656
	Columnas= 14
Cargando fichero-->./raw/MICRODATOS_ACC_VICT_2013/TABLA_VEHIC_2013.csv
	Registros= 155004
	Columnas= 14
Cargando fichero-->./raw/MICRODATOS_ACC_VICT_2014/TABLA_VEHIC_2014.csv
	Registros= 158835
	Columnas= 14
Cargando fichero-->./raw/MICRODATOS_ACC_VICT_2015/TABLA_VEHIC_2015.csv
	Registros= 170749
	Columnas= 14
#### Pers ###
Cargando fichero-->./raw/MICRODATOS_ACC_VICT_2008/TABLA_PERS_24H_2008.csv
	Registros= 223461
	Columnas= 18
Cargando fichero-->.

In [ ]:
df_total.shape

In [41]:
df = pd.DataFrame({'A': ['x', 'y', 'x'], 'B': ['z', 'u', 'z'],
                  'C': ['1', '2', '3'],
                  'D':['j', 'l', 'j']})

In [48]:
pd.get_dummies(df, prefix=['A','B','D'], columns=['A', 'B','D'])

,C,A_x,A_y,B_u,B_z,D_j,D_l
0,1,1,0,0,1,1,0
1,2,0,1,1,0,0,1
2,3,1,0,0,1,1,0


In [131]:
dic_total.


{'Accvict':        ACCIONPEATON  ACCION_PEATON ACERAS  ACOND_CALZADA  ANIO  \
 0               NaN            NaN    NaN            NaN  2008   
 1               NaN            NaN    NaN            NaN  2008   
 2               NaN            NaN    NaN            NaN  2008   
 3               NaN            NaN    NaN            NaN  2008   
 4               NaN            NaN    NaN            NaN  2008   
 5               NaN            NaN    NaN            NaN  2008   
 6               NaN            NaN    NaN            NaN  2008   
 7               NaN            NaN    NaN            NaN  2008   
 8               NaN            NaN    NaN            NaN  2008   
 9               NaN            NaN    NaN            NaN  2008   
 10              NaN            NaN    NaN            NaN  2008   
 11              NaN            NaN    NaN            NaN  2008   
 12              NaN            NaN    NaN            NaN  2008   
 13              NaN            NaN    NaN         

In [ ]:
  df_year=pd.read_csv(files_list[file],delimiter=";",encoding="ISO-8859-1",converters={'COD_MUNICIPIO': str, 'MUNICIPIO': str})


### LECTURA DE LOS DATASET DE VEHICULOS PARA LOS AÑOS 2011-2015

In [165]:
df_year.shape[0]

97756

In [187]:
#Creo un dataframe vacio para posteriormente rellenarlo con los años 2001 al 2015
df_accvict_08_15 = pd.DataFrame()
dict={}
for key in list_keys_dic:
    for file in range(0,len(files_list)):
        df_year=pd.read_csv(files_list[file],delimiter=";",encoding="ISO-8859-1")
        df_year.fillna(0)
        dict[tipology]=files_list[file]
        print("Cargando fichero-->"+files_list[file]+"\n\t"+"Registros= "+str(df_year.shape[0])+"\n\t"+"Columnas= "+str(df_year.shape[1]))
        if tipology=='list_accvict'
        %(tipology)+_08_15=df_accvict_08_15.append(df_year)


Cargando fichero-->./MICRODATOS_ACC_VICT_2008/TABLA_ACCVICT_24H_2008.csv
	Registros= 93161
	Columnas= 32
Cargando fichero-->./MICRODATOS_ACC_VICT_2009/TABLA_ACCVICT_24H_2009.csv
	Registros= 88251
	Columnas= 32
Cargando fichero-->./MICRODATOS_ACC_VICT_2010/TABLA_ACCVICT_24H_2010.csv
	Registros= 85503
	Columnas= 32
Cargando fichero-->./MICRODATOS_ACC_VICT_2011/TABLA_ACCVICT_2011.csv
	Registros= 83027
	Columnas= 39
Cargando fichero-->./MICRODATOS_ACC_VICT_2012/TABLA_ACCVICT_2012.csv
	Registros= 83115
	Columnas= 39
Cargando fichero-->./MICRODATOS_ACC_VICT_2013/TABLA_ACCVICT_2013.csv
	Registros= 89519
	Columnas= 39
Cargando fichero-->./MICRODATOS_ACC_VICT_2014/TABLA_ACCVICT_2014.csv
	Registros= 91570
	Columnas= 39
Cargando fichero-->./MICRODATOS_ACC_VICT_2015/TABLA_ACCVICT_2015.csv
	Registros= 97756
	Columnas= 39


In [189]:
j=df_accvict_08_15.groupby(['ANIO']).size()


In [191]:
type(j)

pandas.core.series.Series

In [ ]:
df_accvict_08_15.groupby(['ANIO']).agg(['mean', 'count'])

In [176]:
print(df_accvict_08_15.head())
#df1=df[['ANIO','TOT_VICTIMAS']]


  ACERAS  ACOND_CALZADA  ANIO CARRETERA  COD_MUNICIPIO  COMUNIDAD_AUTONOMA  \
0      N            0.0  2008    BI-631            NaN                  17   
1      N            0.0  2008      C-12            NaN                   9   
2      N            0.0  2008   TV-7331            NaN                   9   
3      N            0.0  2008     N-340            NaN                   9   
4      N            0.0  2008     CR-26            NaN                   9   

   DENSIDAD_CIRCULACION  DIASEMANA  FACTORES_ATMOSFERICOS   HORA  \
0                   1.0          5                      1  07:00   
1                   1.0          7                      9  14:00   
2                   0.0          7                      1  12:00   
3                   0.0          2                      1  18:00   
4                   1.0          5                      9  19:00   

       ...        TOT_HERIDOS_LEVES30D  TOT_MUERTOS  TOT_MUERTOS30D  \
0      ...                         NaN            0

In [174]:
dict

{0: './MICRODATOS_ACC_VICT_2008/TABLA_ACCVICT_24H_2008.csv',
 1: './MICRODATOS_ACC_VICT_2009/TABLA_ACCVICT_24H_2009.csv',
 2: './MICRODATOS_ACC_VICT_2010/TABLA_ACCVICT_24H_2010.csv',
 3: './MICRODATOS_ACC_VICT_2011/TABLA_ACCVICT_2011.csv',
 4: './MICRODATOS_ACC_VICT_2012/TABLA_ACCVICT_2012.csv',
 5: './MICRODATOS_ACC_VICT_2013/TABLA_ACCVICT_2013.csv',
 6: './MICRODATOS_ACC_VICT_2014/TABLA_ACCVICT_2014.csv',
 7: './MICRODATOS_ACC_VICT_2015/TABLA_ACCVICT_2015.csv'}

In [ ]:
threedee.scatter(df.ANIO, df["TOT_MUERTOS"], df['Volume'])